In [13]:
import datapreparationold, datapreparation220712, comb, extract, scoring, boundaryclone, graph, phylogeny
import pyclonesim, sciclonesim
import EMhard, EMsoft
import scoring
import visualizationsingle, visualizationpair, visualizationsinglesoft
import numpy as np
import pandas as pd
from kneed import KneeLocator
import glob, subprocess, os, Moore

class ResultClass:
    def __init__(self, toollist):
        self.NUM_CLONE_record = pd.DataFrame (  columns = ["TISSUE"] + toollist )
        self.NUM_CLONE_record = self.NUM_CLONE_record.astype (int)
        self.NUM_CLONE_record = self.NUM_CLONE_record.astype ( {"TISSUE" : "str" })

    def acc (self, TISSUE, NUM_CLONE_TABLE, toollist):
        addmatrix = [TISSUE]
        for i in range ( len(NUM_CLONE_TABLE)) :
            addmatrix.append ( NUM_CLONE_TABLE[i] )

        #print (addmatrix)
        # print (  pd.DataFrame (addmatrix, index = ["TISSUE"] + toollist ).transpose()  )
        self.NUM_CLONE_record = self.NUM_CLONE_record.append(  pd.DataFrame (addmatrix, index = ["TISSUE"] + toollist ).transpose()  )


##  Moore1D에 대해서 NUM_CLONE histogram 그리기


In [17]:
import pandas as pd
import numpy as np
import argparse, filetype
import os


def out(command): 
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell=True) 
    return result.stdout.rstrip("\n")


DIR = "/data/project/Alzheimer/EM_cluster/Moore_data/Donor"
DONOR_LIST = glob.glob(DIR + "/*")
DONOR_LIST = [i.split("/")[-1] for i in DONOR_LIST]
k = 0

toollist = ["Moore", "TGIL_soft_1st", "pyclonevi", "sciclone"]
result = ResultClass(toollist)

rowcnt = 0

for DONOR in DONOR_LIST:               # PD42566
    TISSUE_LIST = glob.glob (DIR + "/" + DONOR + "/*")
    TISSUE_LIST = [i.split("/")[-1] for i in TISSUE_LIST]
    for TISSUE in TISSUE_LIST:               # colon_crypt
        SAMPLE_LIST = glob.glob (DIR + "/" + DONOR + "/" + TISSUE + "/*")
        SAMPLE_LIST = [i.split("/")[-1].split(".")[0] for i in SAMPLE_LIST]
        for SAMPLE in SAMPLE_LIST:       # PD42566b_lo00_A7.txt
            INPUT_TSV = DIR + "/" + DONOR + "/" + TISSUE + "/" + SAMPLE + ".txt"
            
            if int(out ("wc -l  " + INPUT_TSV).split(" ")[0]) < 200:       # 200줄이 안 되는 파일은 넘어간다
                continue 


            kwargs = {"INPUT_TSV" : INPUT_TSV,  "NUM_CLONE_TRIAL_START" : 1, "NUM_CLONE_TRIAL_END" : 5, "NUM_CLONE_TRIAL_FORCE" : 1,
                        "RANDOM_PICK":100, "AXIS_RATIO":0, "PARENT_RATIO": 0, "FP_RATIO":0,  "FP_2D" : "False", "TRIAL_NO" : 4, "DEPTH_CUTOFF" : 10, "VERBOSE" : 1,  "MIN_CLUSTER_SIZE" : 5, "NUM_PARENT" : 0, 
                        "NPVAF_DIR" : "", "MYEM_DIR" : "", "SCICLONE_DIR" : "", "PYCLONE_DIR" : "", "PYCLONEVI_DIR" : "",  "COMBINED_OUTPUT_DIR" : "",
                        "KMEANS_CLUSTERNO" : 10, "RANDOM_SEED" : 1, "BENCHMARK_NO" : 10, "MODE" : "Both", "SCORING" : False }

            INPUT_TSV = kwargs["INPUT_TSV"]
            INPUT_FILETYPE, NUM_BLOCK = filetype.main (INPUT_TSV)
            kwargs["NUM_BLOCK_INPUT"], kwargs["NUM_BLOCK"] = NUM_BLOCK, NUM_BLOCK
            SAMPLENAME = INPUT_TSV.split("/")[-1].split(".")[0]     
            kwargs["SAMPLENAME"] = SAMPLE      # 'C10_H9'

            kwargs["NPVAF_DIR"] = "/data/project/Alzheimer/YSscript/EM_MRS/data/npvaf/Moore1D/" + TISSUE + "/" + DONOR + "_" + SAMPLE
            kwargs["MYEM_DIR"] = "/data/project/Alzheimer/YSscript/EM_MRS/data/MyEM/Moore1D/" + TISSUE + "/" + DONOR + "_" + SAMPLE
            kwargs["SCICLONE_DIR"] = "/data/project/Alzheimer/YSscript/EM_MRS/data/sciclone/Moore1D/" + TISSUE + "/" + DONOR + "-" + SAMPLE
            kwargs["PYCLONE_DIR"] = "/data/project/Alzheimer/YSscript/EM_MRS/data/pyclone/Moore1D/" + TISSUE + "/" + DONOR + "-" + SAMPLE
            kwargs["PYCLONEVI_DIR"] = "/data/project/Alzheimer/YSscript/EM_MRS/data/pyclone-vi/Moore1D/" + TISSUE  + "/" + DONOR + "-" + SAMPLE
            kwargs["COMBINED_OUTPUT_DIR"] = "/data/project/Alzheimer/YSscript/EM_MRS/data/combinedoutput/Moore1D/" + TISSUE + "/" + DONOR + "-" + SAMPLE          

            if os.path.isfile(kwargs["COMBINED_OUTPUT_DIR"] + "/TGIL_soft.xiebeni.txt") == True:
                check = 0
                NUM_CLONE_TABLE = [0] * len(toollist)

                for j in range (0, len(toollist)):
                    if os.path.isfile(kwargs["COMBINED_OUTPUT_DIR"] + "/" + toollist[j] + ".mixture.txt") == True:
                        temp = pd.read_csv (kwargs["COMBINED_OUTPUT_DIR"] + "/" + toollist[j] + ".mixture.txt", sep = "\t", header = None)
                        NUM_CLONE_TABLE[j] = temp.shape[1]
                    elif os.path.isfile(kwargs["COMBINED_OUTPUT_DIR"] + "/" + toollist[j] + ".results.txt") == True:
                        with open (kwargs["COMBINED_OUTPUT_DIR"] + "/" + toollist[j] + ".results.txt", "r", encoding = "utf8") as input_results:
                            for line in input_results.readlines():
                                if line.split("\t")[0] == "NUM_CHILD":
                                    NUM_CLONE_TABLE[j] = int(line.split("\t")[1])
                                    break
                        
                    else:
                        check = 1
                        print ("{} 파일 ({})이 없다".format(toollist[j], kwargs["COMBINED_OUTPUT_DIR"] + "/" + toollist[j] + ".txt" ))

                if check == 0:        # 모든 result 파일이 다 있으면 pd에 추가
                    result.acc (TISSUE, NUM_CLONE_TABLE, toollist)
                    rowcnt = rowcnt + 1    
                    k = k  + 1


    

            #os.system (command)
            #print (command)


            if k >= 20:
                break
        if k >= 20:
            break
    if k >= 20:
        break

print (result.NUM_CLONE_record)

                       TISSUE Moore TGIL_soft_1st pyclonevi sciclone
0              skin_epidermis     3             2         2        4
0              skin_epidermis     2             2         1        3
0              skin_epidermis     3             2         2        4
0              skin_epidermis     3             2         1        2
0       oesophagus_epithelium     3             2         3        1
0       oesophagus_epithelium     3             2         2        2
0       oesophagus_epithelium     3             2         1        1
0       oesophagus_epithelium     2             2         2        3
0       oesophagus_epithelium     3             2         3        1
0       oesophagus_epithelium     3             2         1        2
0       oesophagus_epithelium     2             2         2        5
0  small_bowel_brunners_gland     2             1         1        3
0  small_bowel_brunners_gland     1             1         1        1
0  small_bowel_brunners_gland     

##  Moore2D에 대해서 NUM_CLONE histogram 그리기

In [69]:
import pandas as pd
import numpy as np
import argparse, filetype
import os




# 2D 가능한 모든 Moore dataset을 돈다


inputdf = pd.read_csv("/data/project/Alzheimer/EM_cluster/EM_input/summary/Moore_2_sample.txt", sep = "\t")
TISSUE_set = set([])

toollist = ["TGIL_soft_1st", "pyclonevi", "sciclone"]
result = ResultClass(toollist)

rowcnt = 0

for k in range (inputdf.shape[0]):
    DONOR, TISSUE, SAMPLE = inputdf.iloc[k]["DONOR"], inputdf.iloc[k]["TISSUE"], inputdf.iloc[k]["SAMPLE"]
    INPUT_TSV = "/".join(["/data/project/Alzheimer/EM_cluster/EM_input/Moore_2_sample", DONOR, TISSUE, SAMPLE+"_input.txt"])
    
   
    kwargs = {"INPUT_TSV" : INPUT_TSV,  "NUM_CLONE_TRIAL_START" : 1, "NUM_CLONE_TRIAL_END" : 5, "NUM_CLONE_TRIAL_FORCE" : 1,
                "RANDOM_PICK":200, "AXIS_RATIO":0, "PARENT_RATIO": 0, "FP_RATIO":0, "FP_2D" : "False", "TRIAL_NO" : 4, "DEPTH_CUTOFF" : 10, "VERBOSE" : 1,  "MIN_CLUSTER_SIZE" : 5, "NUM_PARENT" : 0, 
                "NPVAF_DIR" : "", "MYEM_DIR" : "", "SCICLONE_DIR" : "", "PYCLONE_DIR" : "", "PYCLONEVI_DIR" : "",  "COMBINED_OUTPUT_DIR" : "", "BENCHMARK_NO" : 10, 
                "KMEANS_CLUSTERNO" : 10, "RANDOM_SEED" : 1, "MODE" : "Both", "SCORING" : False }

    INPUT_TSV = kwargs["INPUT_TSV"]
    INPUT_FILETYPE, NUM_BLOCK = filetype.main (INPUT_TSV)
    kwargs["NUM_BLOCK_INPUT"], kwargs["NUM_BLOCK"] = NUM_BLOCK, NUM_BLOCK
    SAMPLENAME = INPUT_TSV.split("/")[-1].split(".")[0]     
    kwargs["SAMPLENAME"] = SAMPLE      # 'C10_H9'

    kwargs["NPVAF_DIR"] = "/data/project/Alzheimer/YSscript/EM_MRS/data/npvaf/Moore2D/" + TISSUE + "/" + DONOR + "_" + SAMPLE
    kwargs["MYEM_DIR"] = "/data/project/Alzheimer/YSscript/EM_MRS/data/MyEM/Moore2D/" + TISSUE + "/" + DONOR + "_" + SAMPLE
    kwargs["SCICLONE_DIR"] = "/data/project/Alzheimer/YSscript/EM_MRS/data/sciclone/Moore2D/" + TISSUE + "/" + DONOR + "-" + SAMPLE
    kwargs["PYCLONE_DIR"] = "/data/project/Alzheimer/YSscript/EM_MRS/data/pyclone/Moore2D/" + TISSUE + "/" + DONOR + "-" + SAMPLE
    kwargs["PYCLONEVI_DIR"] = "/data/project/Alzheimer/YSscript/EM_MRS/data/pyclone-vi/Moore2D/" + TISSUE  + "/" + DONOR + "-" + SAMPLE
    kwargs["COMBINED_OUTPUT_DIR"] = "/data/project/Alzheimer/YSscript/EM_MRS/data/combinedoutput/Moore2D/" + TISSUE + "/" + DONOR + "-" + SAMPLE



    if os.path.isfile(kwargs["MYEM_DIR"] + "/TGIL_soft_1st.results.txt") == True:
        check = 0
        NUM_CLONE_TABLE = [0] * len(toollist)

        temp = pd.read_csv (kwargs["MYEM_DIR"] + "/TGIL_soft_1st.results.txt", sep = "\t", header = None)
        NUM_CLONE_TABLE[0] = temp.iloc[0,1]

        for j in range (1, len(toollist)):
            if os.path.isfile(kwargs["COMBINED_OUTPUT_DIR"] + "/" + toollist[j] + ".mixture.txt") == True:
                temp = pd.read_csv (kwargs["COMBINED_OUTPUT_DIR"] + "/" + toollist[j] + ".mixture.txt", sep = "\t", header = None)
                NUM_CLONE_TABLE[j] = temp.shape[1]
            else:
                check = 1
                #print ("{} 파일 ({})이 없다".format(toollist[j], kwargs["COMBINED_OUTPUT_DIR"] + "/" + toollist[j] + ".txt" ))

        if check == 0:
            result.acc (TISSUE, NUM_CLONE_TABLE)
            rowcnt = rowcnt + 1
    #else:
        #print ("TGIL_soft_1st 파일 ({})이 없다".format(kwargs["MYEM_DIR"] + "/TGIL_soft_1st.results.txt", ))
        

# result.NUM_CLONE_record

def drawfigure (result, toollist, **kwargs):
    import palettable
    import matplotlib
    import numpy as np
    import seaborn as sns
    import copy

    tabl = palettable.tableau.Tableau_20.mpl_colors
    safe7 = palettable.cartocolors.qualitative.Vivid_10.mpl_colors
    Gr_10 = palettable.scientific.sequential.GrayC_20.mpl_colors

    colorlist = [i for i in safe7]
    sns.set_style("whitegrid")
    #sns.set_palette("tab10")
    sns.set_palette(sns.color_palette(colorlist))

    font_dir = "/home/goldpm1/miniconda3/envs/cnvpytor/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf/"
    font_dirs = matplotlib.font_manager.findSystemFonts(fontpaths=font_dir, fontext='ttf')
    for font in font_dirs:
        matplotlib.font_manager.fontManager.addfont(font)
    matplotlib.rcParams["font.family"] = 'arial'


    # Seaborn을 위해 df를 만들기
    df_melt = pd.melt( result.NUM_CLONE_record, id_vars = "TISSUE", var_name = "toollist", value_name = "NUM_CLONE" )

    matplotlib.pyplot.figure(figsize=(20, 6))
    matplotlib.pyplot.suptitle ("NUM_CLONE ESTIMATION for Moore 2D", fontsize = 30, y = 1, fontweight = "semibold")
    sns.despine()
    sns.swarmplot (data = df_melt, x = "TISSUE", y = "NUM_CLONE", hue = "toollist", dodge = 0.1, size = 10, edgecolor = "gray")
    matplotlib.pyplot.tick_params(axis = 'x', rotation = 45)
    matplotlib.pyplot.yticks ( range (np.min (df_melt["NUM_CLONE"]) , np.max (df_melt["NUM_CLONE"]) + 1  ))


drawfigure (result, toollist)